
# Escape from the Forbidden Corridor



In [2]:

from text_adventure_games import games, actions, things, blocks, viz


## Locations (3)

In [3]:

forbidden_corridor = things.Location(
    "Forbidden Corridor",
    " It's nightime at Hogwarts. A locked door leads into the unknown. Fluffy the three-headed dog naps here."
)
potion_storage = things.Location(
    "Potion Storage Room",
    "Filled with shelves of potions in bottles."
)
trapdoor_room = things.Location(
    "Trapdoor Room",
    "A dim, eerie room where a trapdoor waits."
)


## Items (3)

In [4]:

# Key (hidden under loose floorboard in the corridor)
key = things.Item("key", "a small brass key", "THIS LOOKS LIKE IT MIGHT FIT A LOCK.")
key.add_command_hint("take key")
forbidden_corridor.add_item(key)

# Sleeping Draught
potion = things.Item("potion", "a vial of Sleeping Draught", "SWIRLING LIQUID OF DEEP SLEEP.")
potion.set_property("is_potion", True)
potion.add_command_hint("use potion on fluffy")
potion_storage.add_item(potion)

# Wand Upgrade (enables Lumos)
wand = things.Item("wand", "an upgraded wand", "WILL HELP YOU CAST LUMOS.")
wand.set_property("can_cast_lumos", True)
wand.add_command_hint("cast lumos")
potion_storage.add_item(wand)


## Character

In [5]:

fluffy = things.Character(
    name="fluffy",
    description="A colossal three-headed dog, lightly snoring.",
    persona="I am sleepy but will attack intruders."
)
fluffy.set_property("is_asleep", True)     
fluffy.set_property("is_blocking", True) 
forbidden_corridor.add_character(fluffy)


## Player

In [6]:

player = things.Character(
    name="you",
    description="A student out after curfew.",
    persona="I must get back before Filch catches me."
)


## Connections

In [7]:

forbidden_corridor.add_connection("east", potion_storage) 
forbidden_corridor.add_connection("north", trapdoor_room)

potion_storage.add_connection("west", forbidden_corridor)
trapdoor_room.add_connection("down", forbidden_corridor) 


## Custom Actions

In [8]:

class Unlock_Corridor_Door(actions.Action):
    ACTION_NAME = "unlock door"
    ACTION_DESCRIPTION = "Unlock the corridor door using the brass key"
    ACTION_ALIASES = ["open locked door"]

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)
        self.location = self.parser.get_location(self.character)
        self.k = self.parser.match_item(command, self.character.inventory)

    def check_preconditions(self) -> bool:
        # must be in corridor
        if self.location != forbidden_corridor:
            self.parser.fail("There is no locked door here to unlock.")
            return False
        # need key
        if not self.k or self.k.name != "key":
            self.parser.fail("You need the brass key to unlock the door.")
            return False
        # already unlocked?
        if not forbidden_corridor.get_property("door_locked", True):
            self.parser.fail("The door is already unlocked.")
            return False
        return True

    def apply_effects(self):
        forbidden_corridor.set_property("door_locked", False)
        self.parser.ok("You unlock the door leading east.")


In [9]:

class Use_Potion_On_Fluffy(actions.Action):
    ACTION_NAME = "use potion on fluffy"
    ACTION_DESCRIPTION = "Pour the Sleeping Draught on food for Fluffy"
    ACTION_ALIASES = ["give potion to fluffy", "drug fluffy"]

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)
        self.location = self.parser.get_location(self.character)
        self.p = self.parser.match_item(command, self.character.inventory)

    def check_preconditions(self) -> bool:
        # must be in corridor with fluffy
        if self.location != forbidden_corridor:
            self.parser.fail("Fluffy isn't here.")
            return False
        # must have the potion
        if not self.p or not self.p.get_property("is_potion"):
            self.parser.fail("You don't have the Sleeping Draught.")
            return False
        return True

    def apply_effects(self):
        # consume potion; pacify Fluffy fully
        try:
            self.character.inventory.remove(self.p)
        except Exception:
            pass
        fluffy.set_property("is_asleep", True)
        fluffy.set_property("is_blocking", False)
        self.parser.ok("You quietly pour the draught over Fluffy's dish. He slumps into a deeper sleep.")


In [10]:

class Cast_Lumos(actions.Action):
    ACTION_NAME = "cast lumos"
    ACTION_DESCRIPTION = "Cast Lumos to dispel darkness"
    ACTION_ALIASES = ["lumos"]

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)
        self.location = self.parser.get_location(self.character)
        self.wand = None
        for thing in self.character.inventory:
            if thing.get_property("can_cast_lumos"):
                self.w = thing
                break

    def check_preconditions(self) -> bool:
        if not self.w:
            self.parser.fail("You need a wand capable of casting Lumos.")
            return False
        return True

    def apply_effects(self):
        self.location.set_property("is_dark", False)
        self.parser.ok("Your wand tip glows. The darkness recedes.")


In [11]:

class Open_Trapdoor(actions.Action):
    ACTION_NAME = "open trapdoor"
    ACTION_DESCRIPTION = "Open the trapdoor and escape"
    ACTION_ALIASES = ["open door", "climb down", "go down"]

    def __init__(self, game, command):
        super().__init__(game)
        self.command = command
        self.character = self.parser.get_character(command)
        self.location = self.parser.get_location(self.character)

    def check_preconditions(self) -> bool:
        if self.location != trapdoor_room:
            self.parser.fail("There is no trapdoor here.")
            return False
        if trapdoor_room.get_property("is_dark", True):
            self.game.set_property("lost", True)
            self.parser.fail("You trip and fall into Devil's Snare — GAME OVER.")
            return False
        return True

    def apply_effects(self):
        self.game.set_property("won", True)
        self.parser.ok("You open the trapdoor and climb through."
                       "“You land safely in the passage leading back to Gryffindor Tower. You’ve escaped!”")


## Blocks (Door + Fluffy Guard)

In [ ]:

class Door_Block(blocks.Block):
    def __init__(self, location):
        super().__init__("A locked door bars the way east.", "The door is locked.")
        self.location = location
    def is_blocked(self) -> bool:
        return self.location.get_property("door_locked", True)

class Fluffy_Block(blocks.Block):
    def __init__(self, location, creature):
        super().__init__("Fluffy blocks the way north.", "Fluffy wakes as you approach!")
        self.location = location
        self.creature = creature
    def is_blocked(self) -> bool:
        if self.creature.get_property("is_blocking", True):
            self.game.set_property("lost", True)
            self.parser.fail("Fluffy wakes up and attacks! Game Over.")
            return True
        return False


In [13]:

forbidden_corridor.set_property("door_locked", True)
trapdoor_room.set_property("is_dark", True)

forbidden_corridor.add_block("east", Door_Block(forbidden_corridor))
forbidden_corridor.add_block("north", Fluffy_Block(forbidden_corridor, fluffy))


## Game Class & Loop

In [17]:

class HogwartsMini(games.Game):
    def __init__(self, start_at, player, characters=None, custom_actions=None):
        super().__init__(start_at, player, characters=characters, custom_actions=custom_actions)

    def is_won(self) -> bool:
        if self.get_property("won"):
            self.parser.ok("You win!", self)
            return True
        return False

    def is_lost(self) -> bool:
        return bool(self.get_property("lost"))

# forbidden_corridor.add_hint("You notice a loose floorboard—perhaps something is hidden beneath...")
# potion_storage.add_hint("Among the vials, a label reads 'Sleeping Draught'. Another box holds a wand upgrade.")
# trapdoor_room.add_hint("It's too dark to make out details here. Maybe some light would help.")


## Play

In [20]:
characters = [fluffy]
custom_actions = [Unlock_Corridor_Door, Use_Potion_On_Fluffy, Cast_Lumos, Open_Trapdoor]

game = HogwartsMini(forbidden_corridor, player, characters=characters, custom_actions=custom_actions)

graph = viz.Visualizer(game).visualize()
display(graph)

game.game_loop()


TypeError: get_property() takes 2 positional arguments but 3 were given